In [1]:
# # pip install
# %pip install openai dill seaborn

In [5]:
# # Corpus
# CORPUS = "synthetic_data_corpus"
# # MODEL = "gpt-3.5-turbo-instruct"
# # MODEL = "gpt-4-1106-preview"
# MODEL = "df@meta-llama/Llama-2-70b-chat-hf"


# # The name of the experiment (i.e. where to save the results)
# # EXPERIMENT_NAME = "climate_change_synthetic_k1_with_fewshot_and_both_and_neither"
# from datetime import datetime
# EXPERIMENT_NAME = f"{CORPUS}_{MODEL}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}".replace("/", "_")

# Corpus
# CORPUS = "synthetic_data_corpus"
# CORPUS = "converted_climate_change_gpt-4-turbo-preview_gpt-4-turbo-preview_10_10"
# CORPUS = "/workspaces/dev/projects/narratives/synthetic/corpora/climate_change/20_10.json"
CORPUS_NAME = "climate_change"
# MODEL = "gpt-3.5-turbo"
# MODEL = "gpt-3.5-turbo-0125"
# MODEL = "gpt-4-turbo-preview"
MODEL = "df@meta-llama/Llama-2-70b-chat-hf"

# MODE = "seeds"
# MODE = "distilled"
MODE = "summaries"
# MODE = "zero-shot"

# The name of the experiment (i.e. where to save the results)
# EXPERIMENT_NAME = "climate_synthetic_test_k2"
# EXPERIMENT_NAME = CORPUS_NAME + "_" + MODEL + "_gpt"
EXPERIMENT_NAME = "test_ignore_me"

CORPUS_SIZE = "/2_2"

In [6]:
CORPUS = (
    "/workspaces/dev/projects/narratives/synthetic/corpora/"
    + CORPUS_NAME
    # + "/20_10.json"
    # + "/"
    + CORPUS_SIZE
    + ".json"
)

In [10]:
from soda.openai.text import completion_model_batched, instruct_chat_model_batched


def get_classifications(
    texts, classes, model, batch_size=1, num_threads=1, max_tokens=1
):
    # if "gpt-4" in model or model == "gpt-3.5-turbo" or model == "gpt-3.5-turbo-0125":
    #     return get_classifications_gpt4(
    #         texts, classes, model, batch_size, num_threads, max_tokens
    #     )
    # else:
    #     return get_classifications_instruct(
    #         texts, classes, model, batch_size, num_threads, max_tokens
    #     )

    return get_classifications_gpt4(
        texts, classes, model, batch_size, num_threads, max_tokens
    )


def get_classifications_gpt4(
    texts, classes, model, batch_size=1, num_threads=1, max_tokens=1
):
    """
    Get the classification probabilities using the GPT-4 chat API
    """

    # Construct the system message
    system_message = """
You are a classification system. You will be given some text and you must respond with a single class that the text most likely belongs to. Ensure your response is only the class, with no other text. These are the classes you can choose from:
""".strip()
    system_message += "\n\n"
    for class_ in classes:
        system_message += f"*** Class '{class_}' ***\n"
        system_message += classes[class_]
        system_message += "\n"

    # Map the texts to the prompts
    # prompts = [
    #     # "Text: " + text + "\nClass (from { " + ", ".join(classes.keys()) + " }):"
    #     # "This is the text: "
    #     + text
    #     # + "\n\n"
    #     # + "Respond with the class that best fits the text, from these classes:
    #     # + ", ".join(classes.keys())
    #     for text in texts
    # ]
    prompts = texts

    # # Print a summary of the prompts
    # for prompt in prompts:
    #     print("--- System Message ---")
    #     print(system_message)
    #     print()

    #     print("--- Prompt ---")
    #     print(prompt)
    #     print()
    #     print()

    # Get the completions
    results = instruct_chat_model_batched(
        system_message,
        # texts,
        prompts,
        model,
        num_threads=num_threads,
        logprobs=True,
        top_logprobs=min(5, len(classes)),
        temperature=0.0,
        timeout=5,
    )

    # Get the token logprobs for each completion
    responses = [result.choices[0].logprobs.content[0] for result in results]
    usages = [result.usage for result in results]

    # For each class, get the logprobs for each token
    # Use -inf for classes without an entry in the top logprobs
    logprobs = []
    for response in responses:
        classification = {class_: -float("inf") for class_ in classes}

        # Get the logprobs for each token
        # for token, logprob in response.items():
        for top_logprob in response.top_logprobs:
            # Get the class for this token
            class_ = top_logprob.token.strip()

            # Update the classification
            classification[class_] = top_logprob.logprob

        logprobs.append(classification)

    return logprobs, usages


# def get_classifications_instruct(
#     texts, classes, model, batch_size=1, num_threads=1, max_tokens=1
# ):
#     """
#     Get the classification probabilities for each text and class.
#     """

#     # Construct the prompt
#     prompt = ""
#     prompt += "You are a classification system. Given the text below, you will classify it into either of the given classes. Output the class name.\n"
#     # prompt += "You are a classification system. Given the text below, you will classify it into one of the following classes. If no class is suitable, choose \"none\". If more than one class is suitable, choose \"all\". Ensure you match the classes exactly, including wording and casing.\n"
#     # for class_ in classes:
#     #     prompt += f"- {class_}\n"
#     # for class_, description in classes.items():
#     #     prompt += f"- {class_}: {description}\n"
#     # prompt += "*** Class 'a' ***\n"
#     # prompt += classes["a"]
#     # prompt += "\n"

#     # prompt += "*** Class 'b' ***\n"
#     # prompt += classes["b"]
#     # prompt += "\n"

#     for class_ in classes:
#         prompt += f"*** Class '{class_}' ***\n"
#         prompt += classes[class_]
#         prompt += "\n"

#     prompt += "\nText: {TEXT}\nClass (from { " + ", ".join(classes.keys()) + " }):"

#     # Create the prompts for each text
#     prompts = [prompt.format(TEXT=text) for text in texts]

#     # Get the completions
#     completions = completion_model_batched(
#         prompts,
#         model,
#         batch_size=batch_size,
#         num_threads=num_threads,
#         max_tokens=max_tokens,
#         logprobs=2,
#     )
#     # print(completions)

#     usages = [completion.usage for completion in completions]

#     # Get the token logprobs for each completion
#     logprobs = []
#     # For each class, get the logprobs for each token
#     # Use -inf for classes without an entry in the top logprobs
#     for completion in completions.choices:
#         classification = {class_: -float("inf") for class_ in classes}

#         # Get the logprobs for each token
#         for token, logprob in completion.logprobs.top_logprobs[0].items():
#             # Get the class for this token
#             class_ = token.strip().replace("▁", "")

#             # Update the classification
#             # classification[class_] = logprob

#             # There might be duplicates due to the ▁ character that DF seems to use
#             classification[class_] = max(
#                 classification.get(class_, -float("inf")), logprob
#             )

#         logprobs.append(classification)

#     return logprobs, usages


# Classify a simple dataset about cats and dogs
# texts = ["This is a cat", "This is a dog", "This is a cat, and this is a dog"]
texts = [
    "This is a cat",
    "This is a dog",
    "This is a cat, and this is a dog",
    "We're not talking about cats and dogs here!",
]
# classes = ["cat", "dog"]
classes = {
    "cat": "Discusses a cat",
    "dog": "Discusses a dog",
}

classifications, usages = get_classifications(
    texts, classes, MODEL, batch_size=1, num_threads=2, max_tokens=1
)

print(classifications)
prompt_tokens = sum(usage.prompt_tokens for usage in usages)
completion_tokens = sum(usage.completion_tokens for usage in usages)
print("Prompt tokens:", prompt_tokens)
print("Completion token:", completion_tokens)

100%|██████████| 4/4 [00:00<00:00, 2863.01it/s]

[ChatCompletion(id='chatcmpl-8b753fc646c94cff8425a1a1df862267', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='  cat', role='assistant', function_call=None, tool_calls=None, name=None))], created=1708565309, model='meta-llama/Llama-2-70b-chat-hf', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=3, prompt_tokens=105, total_tokens=108)), ChatCompletion(id='chatcmpl-44184bbfb9f44b609b618552dc61287d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='  dog', role='assistant', function_call=None, tool_calls=None, name=None))], created=1708565309, model='meta-llama/Llama-2-70b-chat-hf', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=3, prompt_tokens=105, total_tokens=108)), ChatCompletion(id='chatcmpl-7238cc1cc15e4fc9803d31abe9fa271c', choices=[Choice(finish_reason='stop', index=0, logprobs=None, messag

AttributeError: 'NoneType' object has no attribute 'content'

In [ ]:
# Multiprocessing with joblib
import os
import yaml
from tqdm.auto import tqdm
import random


# # Load texts from corpora/{CORPUS}.json
# with open(os.path.join("../corpora", f"{CORPUS}.json"), "r") as f:
#     corpus_data = yaml.safe_load(f)

with open(CORPUS, "r") as f:
    corpus_data = yaml.safe_load(f)

seeds = corpus_data["seeds"]
distilled = corpus_data["distilled"]
summarized = corpus_data["summarized"]
names = corpus_data["names"]
dataset = corpus_data["dataset"]

# Convert the dataset to corpus data format
corpus_data = []
for seed_set in dataset:
    for a in seed_set["a_first"]["a"]:
        corpus_data.append({"text": a, "speakername": "a"})
    for b in seed_set["a_first"]["b"]:
        corpus_data.append({"text": b, "speakername": "b"})
    for a in seed_set["b_first"]["a"]:
        corpus_data.append({"text": a, "speakername": "a"})
    for b in seed_set["b_first"]["b"]:
        corpus_data.append({"text": b, "speakername": "b"})

# Shuffle
random.shuffle(corpus_data)

print("Loaded {} texts from corpus".format(len(corpus_data)))
print("Total word count:", sum([len(text["text"].split()) for text in corpus_data]))

# Create the training data
X = [text["text"] for text in corpus_data]
y = [text["speakername"] for text in corpus_data]
print(len(X), len(y))

# classes = {
#     "a": "A helpful statement.",
#     "b": "An unhelpful statement.",
# }

# Construct the classes depending on the mode
a_desc = ""
b_desc = ""
if MODE == "seeds":
    a_desc = "\n"
    b_desc = "\n"
    for seed in seeds:
        a_desc += "  " + seed["a"] + "\n"
        b_desc += "  " + seed["b"] + "\n"
elif MODE == "distilled":
    a_desc = "\n"
    b_desc = "\n"
    for seed in distilled:
        a_desc += "  " + seed["a"] + "\n"
        b_desc += "  " + seed["b"] + "\n"
elif MODE == "summaries":
    a_desc = summarized["a"]
    b_desc = summarized["b"]
elif MODE == "zero-shot":
    a_desc = "A helpful statement."
    b_desc = "An unhelpful statement."
else:
    raise ValueError(f"Unknown mode: {MODE}")

# Define the classes
# classes = {
#     "a": a_desc,
#     "b": b_desc,
# }
classes = {
    names["a"]: a_desc,
    names["b"]: b_desc,
}

# Get the classifications
classifications, usages = get_classifications(
    # X, classes, MODEL, batch_size=20, num_threads=50, max_tokens=1
    X,
    classes,
    MODEL,
    batch_size=1,
    # num_threads=50,
    num_threads=20,
    max_tokens=1,
)

In [ ]:
# Get the usage
prompt_tokens = sum(usage.prompt_tokens for usage in usages)
completion_tokens = sum(usage.completion_tokens for usage in usages)
print("Prompt tokens:", prompt_tokens)
print("Completion token:", completion_tokens)

# Estimate cost given the model
c_prompt = -1
c_completion = -1
if "gpt-4" in MODEL:
    c_prompt = 0.01 / 1000
    c_completion = 0.03 / 1000
elif "gpt-3.5-turbo" in MODEL:
    c_prompt = 0.0005 / 1000
    c_completion = 0.0015 / 1000

# Estimate cost
cost = c_prompt * prompt_tokens + c_completion * completion_tokens
if cost >= 0:
    print("Cost estimate: ${:.3f}".format(cost))
else:
    print("No cost estimate available")

In [ ]:
# Save the data in CSV format
data = []
total = 0
correct = 0

# Generic
incorrect_map = {class_: 0 for class_ in classes}
correct_map = {class_: 0 for class_ in classes}

a_name = names["a"]
b_name = names["b"]

for text, truth, classification in tqdm(zip(X, y, classifications), total=len(X)):
    # print(text[:20] + "...", truth, classification)
    # print("*** Datum ***")
    # print("  Text: " + text)
    # print("  Ground Truth: " + truth)
    # print("  Classification: " + str(classification))
     
    # # Add the data
    # data.append(
    #     {
    #         "text": text,
    #         # "myth": classification["myth"],
    #         "myth": min(
    #             classification.get("myth", -float("inf")),
    #             classification.get("my", -float("inf")),
    #         ),
    #         "science": classification["science"],
    #         "classification": max(classification, key=classification.get),
    #         "truth": truth,
    #     }
    # )

    # Map truth to the name
    if truth == "a":
        truth = a_name
    elif truth == "b":
        truth = b_name
    else:
        raise ValueError(f"Unknown truth: {truth}")

    # print("  Ground Truth (Name): " + truth)
    
    # Add the data (class-agnostic)
    datum = {
        "text": text,
        "truth": truth,
        "classification": max(classification, key=classification.get),
    }
    # datum.update(classification)
    data.append(datum)

    # # if max(classification, key=classification.get) == truth:
    # if data[-1]["classification"] == truth:
    #     correct += 1
    #     correct_map[truth] += 1
    # else:
    #     incorrect_map[truth] += 1

    # The classification will be a prefix of one of the classes
    # Find the class with that prefix (throw an error if there's more than one match)
    # maximum = max(classification, key=classification.get)

    # Find the match
    # match = None
    # for class_ in classes:
    #     if class_.startswith(classification):
    #         if match is not None:
    #             raise ValueError(f"Multiple matches for {classification}")
    #         match = class_

    matches = {}
    for class_ in classes:
        matches[class_] = float("-inf")
        for token in classification:
            # print("Comparing", class_, "with", token)
            # print(f"Comparing '{class_}' with '{token}'")
            if class_.startswith(token):
                # print("  Found match!")
                matches[class_] = max(matches[class_], classification[token])

    # print(f"  Truth: {truth}, Prediction: {max(matches, key=lambda k: matches[k])}")
    # print(f"  Matches:")
    # for class_ in matches:
    #     print(f"    {class_}: {matches[class_]}")

    # Get the best match
    # print("Matches:", matches)
    match = max(matches, key=lambda k: matches[k])

    # Update the datum
    data[-1]["classification"] = match

    # Update the counts
    if match == truth:
        correct += 1
        correct_map[truth] += 1
    else:
        incorrect_map[truth] += 1

    # Assign the match to the classification
    total += 1

# Show confidence matrix
print("Correct:", correct_map)
print("Incorrect:", incorrect_map)

In [ ]:
# Create the directory
import os
directory = os.path.join("corpus_results", "gpt", MODEL, CORPUS_NAME, MODE + "/")

# Create the directory
os.makedirs(directory, exist_ok=True)


# Save to CSV
import pandas as pd

df = pd.DataFrame(data)
df.to_csv(os.path.join("corpus_results", f"{EXPERIMENT_NAME}.csv"), index=False)

# Show accuracy and incorrect counts
print("Accuracy:", correct / total)
print("Incorrect counts:", incorrect_map)

# Show a confusion matrix as a pandas df
import pandas as pd

# Get the confusion matrix
confusion_matrix = pd.crosstab(
    df["truth"], df["classification"], rownames=["Truth"], colnames=["Classification"]
)

print(confusion_matrix)

# Convert to percentages
confusion_matrix = confusion_matrix / confusion_matrix.sum(axis=0)
print(confusion_matrix)

# Save confusion matrix and data to corpus_results/classifiers/{EXPERIMENT_NAME}
import os
import shutil
import yaml

# Save the confusion matrix
confusion_matrix.to_csv(os.path.join(directory, "confusion_matrix.csv"))

# Save the data
df.to_csv(os.path.join(directory, "data.csv"), index=False)

# Save the usage data as a CSV
usage_data = []
for usage in usages:
    usage_data.append(
        {
            "prompt_tokens": usage.prompt_tokens,
            "completion_tokens": usage.completion_tokens,
        }
    )

usage_df = pd.DataFrame(usage_data)
usage_df.to_csv(os.path.join(directory, "usage.csv"), index=False)